In [2]:
%useLatestDescriptors
%use dataframe, serialization, kandy

In [1]:
val ktor_version = "2.3.8"

USE {
    dependencies {
        implementation("io.ktor:ktor-client-cio-jvm:$ktor_version")
        implementation("io.ktor:ktor-client-content-negotiation-jvm:$ktor_version")
        implementation("io.ktor:ktor-serialization-kotlinx-json-jvm:$ktor_version")
    }
}

In [3]:
import io.ktor.client.*
import io.ktor.client.engine.cio.*
import io.ktor.client.plugins.contentnegotiation.*
import io.ktor.client.plugins.contentnegotiation.*
import io.ktor.serialization.kotlinx.json.*

val client = HttpClient(CIO){
    install(ContentNegotiation){
        json(Json { 
            ignoreUnknownKeys = true
         })
    }
}

In [4]:
val kotlinChannelId = "UCP7uiEZIqci43m22KDl0sNw"
val API_KEY = System.getenv("YOUTUBE_API_KEY")
val apiUrl = "https://www.googleapis.com/youtube/v3"


In [6]:
@Serializable
data class SearchListResponse(
    val items: List<SearchResult>,
    val nextPageToken: String? = null
)

@Serializable
data class SearchResult(
    val id: VideoId
)

@Serializable
data class VideoId(
    val videoId: String
)

In [7]:
import io.ktor.client.call.*
import io.ktor.client.request.*
import kotlinx.coroutines.runBlocking

var ids = mutableListOf<String>()

runBlocking {
    var nextPageToken: String? = null
    do {
        // Request a page of videos
        val searchResponse: SearchListResponse = client.get("$apiUrl/search") {
            parameter("part", "id")
            parameter("channelId", kotlinChannelId)
            parameter("maxResults", "50")
            parameter("type", "video")
            parameter("key", API_KEY)
            if (nextPageToken != null) {
                parameter("pageToken", nextPageToken)
            }
        }.body()

        // Print the video IDs
        for (searchResult in searchResponse.items) {
            ids.add(searchResult.id.videoId)
        }

        // Move on to the next page
        nextPageToken = searchResponse.nextPageToken
    } while (nextPageToken != null)
}

ids

[f73UGMnoR30, 11Pu6RMMEg8, Vuv6EAU_uSg, D2tNpcRULKQ, VdRb3FVD914, atj5HxEQNfA, WafrOiD5PvY, m4Cqz2_P9rI, LH-mOKxA4w4, 5_W5YKPShZ4, 6-XSehwRgSY, _3ErSoKsoNQ, 2PLYlDJPelQ, 4-qOxvjjF8g, o4emra1xm88, QvtmbYfkOO0, 54WEfLKtCGk, j_LEcry7Pms, L8aFK7QrbA8, zluKcazgkV4, 0jWo3o7r-W4, pXtTp4Uxuhk, iyEWXyuuseU, TCmrY4HXUj4, L9wqYQ-fXaM, QGHRAQMj4Lw, zu1PUAvk_Lw, -kltG4Ztv1s, d_Mor21W_60, LpqvtgibbsQ, 8FUHKx58t4c, iTdJJq_LyoY, 0JRPA0tt9og, cLyTx5wSPbg, MyvJ7G6aErQ, ApXbm1T_eI4, EVLnWOcR0is, CQlBQ5tfbHE, C9gCm51RhsU, ovAqcwFhEGc, NB8MOZxzxic, 8F19ds109-o, db19VFLZqJM, ihMhu3hvCCE, IL3RLKvWJF4, maTL7Whco70, cpagMN1H9_g, wwplVknTza4, tukZzLdc_dM, dPhZIo27fYE, pYK5KkuZ3aU, x2bZJv8i0vw, 9fVoqEFWTNI, mye9NjvxVSU, R1JpkpPzyBU, i-kyPp1qFBA, atV8liVgd3w, 698I_AH8h6s, R2395u7SdcI, E9hZITOFCTQ, bv-VyGM3HCY, Vg_atICXpLk, aW1iR0Mmitk, m1pmqE2oHpI, xka7k7RUfWI, Ed3t4WAe0Co, JC9P186sY6Y, O4u80a2Qh_k, 6EQd_SDR6n0, SVlY7Mca1xg, BXzkaR2-DEg, ECOf0PeSANw, zDRlEp7r5i0, 49_eQ395QbE, 4BgZhHcyhY8, hXfGybzWaiA, oUdKTlWchT0

In [8]:
val df = dataFrameOf("id" to ids)

In [9]:
import io.ktor.client.statement.*
import kotlinx.coroutines.delay

val withStats =
    df
        .add("stats") {// Step 2: For each video ID, get the video's statistics
            runBlocking {
                DataFrame.readJsonStr(client.get("$apiUrl/videos") {
                    parameter("part", "statistics")
                    parameter("id", id)
                    parameter("key", API_KEY)
                }.bodyAsText())
            }
        }
        .add("snippet") {// Step 2: For each video ID, get the video's statistics
            runBlocking {
                DataFrame.readJsonStr(client.get("$apiUrl/videos") {
                    parameter("part", "snippet")
                    parameter("id", id)
                    parameter("key", API_KEY)
                }.bodyAsText())
            }
        }

In [10]:
val df1 = withStats.explode { stats and snippet }

In [11]:
val df2 = df1.explode { stats.items and snippet.items }

In [12]:
val withStatsOnTop = df2.move { stats.items.statistics }.toTop()
        .move{snippet.items.snippet named "video"}.toTop()
        .remove{stats and snippet}
        .parse()


In [13]:
withStatsOnTop


<iframe onload="o_resize_iframe_out_1()" style="width:100%;" class="result_container" id="iframe_out_1" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-1644167168"></table>

<p class="dataframe_description">... showing only top 20 of 328 rows</p><p class="dataframe_description">DataFrame: rowsCount = 328, columnsCount = 3</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for 

In [14]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalTransform
import org.jetbrains.letsPlot.Geom
import org.jetbrains.letsPlot.annotations.layerLabels

withStatsOnTop.plot {
    points {
        x(statistics.viewCount){scale=continuous(transform = Transformation.LOG10)}
        y(statistics.likeCount)
        size(statistics.commentCount) {legend.name = "Number of comments"}
        tooltips(video.title)
        color(statistics.commentCount) { scale = continuousColorHue(); legend.type = LegendType.None }
        layout {
            title = "Kotlin Channel Youtube Stats"
            xAxisLabel = "Number of views"
            yAxisLabel = "Number of likes"
        }
    }
}

<head>
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.0.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="WqqKBZ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Kotlin Channel Youtube Stats"
},
"mapping":{
},
"data":{
"likeCount":[677.0,824.0,105.0,90.0,58.0,54.0,169.0,703.0,363.0,2817.0,371.0,238.0,4471.0,138.0,481.0,251.0,890.0,801.0,734.0,769.0,233.0,435.0,222.0,206.0,786.0,188.0,861.0,268.0,355.0,416.0,145.0,668.0,402.0,483.0,277.0,974.0,208.0,87.0,45.0,680.0,574.0,498.0,207.0,254.0,891.0,579.0,257.0,770.0,220.0,185.0,742.0,736.0,53.0,555.0,484.0,1758.0,16.0,421.0,859.0,87.0,242.0,19.0,592.0,167.0,243.0,498.0,536.0,56.0,97.0,35.0,42.0,628.0,44.0,734.0,72.0,51.0,371.0,18.0,41.0,818.0,225.0,40.0,19.0,32.0,661.0,60.0,465.0,68.0,80.0,480.0,1153.0,40.0,307.0,247.0,288.0,52.0,304.0,535.0,248.0,1262.0,1262.0,207.0,118.0,458.0,1189.0,49.0,299.0,322.0,206.0,234.0,803.0,139.0,396.0,88.0,730.0,24.0,112.0,59.0,113.0,107.0,401.0,157.0,315.0,1281.0,873.0,1253.0,86.0,391.0,40.0,143.0,166.0,167.0,260.0,44.0,37.0,64.0,127.0,1005.0,456.0,325.0,65.0,47.0,159.0,114.0,1230.0,62.0,71.0,44.0,494.0,189.0,315.0,62.0,260.0,388.0,275.0,961.0,643.0,696.0,114.0,27.0,118.0,185.0,22.0,54.0,206.0,657.0,739.0,55.0,70.0,154.0,126.0,20.0,18.0,218.0,151.0,166.0,406.0,295.0,949.0,92.0,28.0,50.0,37.0,190.0,313.0,15.0,348.0,250.0,198.0,61.0,354.0,96.0,66.0,43.0,214.0,315.0,603.0,88.0,141.0,163.0,125.0,67.0,930.0,783.0,184.0,452.0,107.0,127.0,696.0,103.0,96.0,127.0,390.0,106.0,618.0,79.0,119.0,116.0,198.0,96.0,89.0,283.0,713.0,128.0,134.0,357.0,143.0,650.0,58.0,146.0,166.0,168.0,606.0,273.0,51.0,107.0,21.0,66.0,165.0,668.0,121.0,258.0,38.0,104.0,173.0,49.0,86.0,438.0,116.0,150.0,143.0,273.0,51.0,331.0,250.0,165.0,442.0,110.0,166.0,144.0,47.0,68.0,141.0,606.0,126.0,463.0,340.0,299.0,192.0,40.0,153.0,301.0,59.0,183.0,260.0,14.0,174.0,209.0,211.0,97.0,48.0,85.0,262.0,566.0,42.0,98.0,77.0,137.0,31.0,383.0,237.0,24.0,194.0,132.0,96.0,262.0,48.0,58.0,71.0,54.0,160.0,268.0,132.0,70.0,153.0,59.0,76.0,720.0,183.0,14.0,191.0,209.0,85.0,98.0,77.0,237.0,32.0,31.0,24.0,194.0,71.0,109.0,355.0,72.0,142.0,123.0,466.0,164.0],
"viewCount":[19496.0,10834.0,2851.0,3201.0,3398.0,1548.0,5315.0,18398.0,6069.0,108850.0,12436.0,6460.0,98966.0,5561.0,14617.0,7640.0,61159.0,14988.0,13549.0,25015.0,7544.0,17307.0,4881.0,6489.0,14338.0,5145.0,18535.0,5810.0,11403.0,11571.0,6497.0,25361.0,10513.0,14626.0,8779.0,18059.0,6837.0,2318.0,1331.0,18169.0,10563.0,14190.0,11062.0,8174.0,12595.0,16268.0,6715.0,19286.0,4901.0,5828.0,19708.0,13498.0,2110.0,14504.0,12245.0,37911.0,654.0,11210.0,13252.0,2605.0,10811.0,1304.0,18814.0,2744.0,4604.0,16419.0,5752.0,1708.0,3936.0,1179.0,1382.0,11399.0,1711.0,12804.0,2073.0,1726.0,6552.0,754.0,2123.0,26634.0,7180.0,1389.0,790.0,1077.0,22488.0,7452.0,13929.0,2600.0,2735.0,10469.0,48258.0,1294.0,10561.0,8573.0,10285.0,30880.0,11413.0,6811.0,6619.0,56145.0,56145.0,5736.0,3119.0,24576.0,31948.0,2177.0,9257.0,3648.0,5763.0,4748.0,25816.0,3566.0,9849.0,2367.0,21165.0,1161.0,4348.0,2554.0,2649.0,2768.0,13209.0,4970.0,6277.0,70293.0,27123.0,25100.0,2036.0,11395.0,1646.0,5319.0,6002.0,10109.0,9325.0,1456.0,1730.0,1949.0,3737.0,49650.0,9184.0,14942.0,3441.0,1297.0,4771.0,3322.0,36723.0,2510.0,3383.0,2192.0,19309.0,5668.0,15864.0,1986.0,5432.0,10564.0,14252.0,24901.0,50229.0,14981.0,5804.0,1141.0,9908.0,6505.0,0.0,1916.0,8751.0,18288.0,15350.0,1256.0,2475.0,4518.0,4562.0,1150.0,745.0,4973.0,4968.0,5017.0,13851.0,10838.0,50970.0,4054.0,889.0,2121.0,1512.0,3872.0,15371.0,726.0,13887.0,9400.0,8945.0,4560.0,14677.0,4116.0,2186.0,1047.0,12735.0,8478.0,12600.0,3859.0,6571.0,6516.0,3552.0,2940.0,14754.0,17198.0,8385.0,17491.0,3799.0,4211.0,26265.0,3994.0,4119.0,6680.0,16680.0,2490.0,15189.0,2033.0,6385.0,4044.0,6265.0,3595.0,3349.0,10188.0,20979.0,3460.0,2873.0,9109

In [15]:
fun imageLink(imgUrl: URL, linkurl: String) = 
    HTML("<a href = $linkurl>" + IMG(imgUrl, width = 100) + "</a>")

# Most liked

In [22]:
HTML(
withStatsOnTop.sortByDesc { statistics.likeCount }.head(5)
    .remove { statistics.favoriteCount }
    .add("url") { imageLink(video.thumbnails.standard.url, "https://www.youtube.com/watch?v=" + id) }
    .select { "url" and video.title and statistics }
    .ungroup { all() }
    .select("url")
    .toHTML().toString())

url
"{ ""data"": { ""text/html"": ..."
"{ ""data"": { ""text/html"": ..."
"{ ""data"": { ""text/html"": ..."
"{ ""data"": { ""text/html"": ..."
"{ ""data"": { ""text/html"": ..."


# Most commented

In [42]:
withStatsOnTop.sortByDesc { statistics.commentCount }.head(5)
    .remove { statistics.favoriteCount }
    .add("url") { imageLink(video.thumbnails.standard.url, "https://www.youtube.com/watch?v=" + id) }
    .select { "url" and video.title and statistics }
    .ungroup { all() }

<iframe onload="o_resize_iframe_out_11()" style="width:100%;" class="result_container" id="iframe_out_11" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-1929379830"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 5</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0

# Most viewed

In [43]:
import io.ktor.http.*

withStatsOnTop.sortByDesc { statistics.viewCount }.head(5)
    .remove { statistics.favoriteCount }
    .add("url") { imageLink(video.thumbnails.standard.url, "https://www.youtube.com/watch?v=" + id) }
    .select { "url" and video.title and statistics }
    .ungroup { all() }

<iframe onload="o_resize_iframe_out_12()" style="width:100%;" class="result_container" id="iframe_out_12" frameBorder="0" srcdoc=" <html>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}




:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 
<table class="dataframe" id="df_-1929379829"></table>

<p class="dataframe_description">DataFrame: rowsCount = 5, columnsCount = 5</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (let i = 0